In [1]:
import pandas as pd
import numpy as np

postal_codes = pd.read_csv("postal_codes.csv", dtype={'Zip':str})
users = pd.read_csv("users.tsv", sep = "\t", dtype={'postal_code':str, 'phone_number':str}).replace({np.nan: None})
items = pd.read_csv("items.tsv", sep = "\t", dtype={'piece_count':pd.Int64Dtype()}).replace({np.nan: None})
swaps = pd.read_csv("swaps.tsv", sep = "\t").replace({np.nan: None})

In [2]:
import mysql.connector as mysql

# Connect to the DB
connection = mysql.connect(
    user='loader',
    password='secret',
    database='GameSwap',
    host='localhost'
)
cursor = connection.cursor()


In [3]:
# Postal codes
for i, p in postal_codes.iterrows():
    sql = """INSERT INTO Address (postal_code, city, state, latitude, longitude) 
            VALUES (%s,%s,%s,%s,%s)"""
    val = (p.Zip, p.City, p.State, p.Latitude, p.Longitude)
    cursor.execute(sql, val)

In [4]:
# Insert users and phones
for i, u in users.iterrows():
    sql = """INSERT INTO User (email, password, first_name, last_name, nickname, postal_code) 
            VALUES (%s,%s,%s,%s,%s,%s)"""
    val = (u.email, u.password, u.first_name, u.last_name, u.nickname, u.postal_code)
    cursor.execute(sql, val)
    if not pd.isna(u.phone_number):
        sql = """INSERT INTO Phone (phone_number, user_email, phone_type, share) 
                VALUES (%s,%s,%s,%s)"""
        val = (f"{u.phone_number[0:3]}-{u.phone_number[3:6]}-{u.phone_number[6:10]}",
               u.email, u.phone_type, u.to_share)
        cursor.execute(sql, val)

In [5]:
# Insert Items
sql = """INSERT INTO VideoPlatform (platform) VALUES ('Nintendo')"""
cursor.execute(sql)

sql = """INSERT INTO VideoPlatform (platform) VALUES ('PlayStation')"""
cursor.execute(sql)

sql = """INSERT INTO VideoPlatform (platform) VALUES ('Xbox')"""
cursor.execute(sql)

for i, item in items.iterrows():
    sql = """INSERT INTO Item (item_number, name, `condition`, description, user_email) VALUES (%s,%s,%s,%s,%s)"""
    val = (item.item_number, item.title, item.condition, item.description, item.email)
    cursor.execute(sql, val)
    if item.type == "Board Game":
        sql = """INSERT INTO BoardItem (item_number) VALUES (%s)"""
        val = (item.item_number,)
    elif item.type == "Card Game":
        sql = """INSERT INTO CardItem (item_number) VALUES (%s)"""
        val = (item.item_number,)
    elif item.type == "Video Game":
        sql = """INSERT INTO VideoItem (item_number, platform, media) VALUES (%s,%s,%s)"""
        val = (item.item_number, item.platform, item.media)
    elif item.type == "Computer Game":
        sql = """INSERT INTO ComputerItem (item_number, platform) VALUES (%s,%s)"""
        val = (item.item_number, item.platform)
    elif item.type == "Jigsaw Puzzle":
        sql = """INSERT INTO JigsawItem (item_number, piece_count) VALUES (%s,%s)"""
        val = (item.item_number, item.piece_count)
        
    cursor.execute(sql, val)

In [6]:
# Insert Swaps
for i, s in swaps.iterrows():
    sql = """INSERT INTO Swap (proposed_item_number, desired_item_number, proposed_date, accept_reject_date, status,
            proposer_rating, counterparty_rating) 
            VALUES (%s,%s,%s,%s,%s,%s,%s)"""
    val = (s.item_proposed, s.item_desired, s.date_proposed, s.date_reviewed, s.accepted, 
           s.proposer_rate, s.counterparty_rate)
    cursor.execute(sql, val)

In [7]:
connection.commit()
connection.close()

In [8]:
connection.close()